# Web Scraing MahaRera website to Extact information of Registered Agents In Mumbai City Using Selenium Web Driver and Beautiful Soup For Parsing HTML in python


In [165]:
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:14 http://ppa.launchpad.net/graph

In [166]:
!pip install BeautifulSoup4
! pip install requests

In [167]:
# Importing Required Library
import selenium
from selenium import webdriver
from selenium.webdriver.support.select import Select
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',options=options)
driver =webdriver.Chrome('chromedriver',options=options)
driver.implicitly_wait(20)


import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from PIL import Image



In [168]:
driver.maximize_window() # For maximizing window
driver.implicitly_wait(20)

## Funtion to Extract Required Rows and Columns from Table present in Pandas dataframe 

In [169]:
def tableDataText(table):       
    rows = []
    trs = table.find_all('tr')
    headerow = [td.get_text(strip=True) for td in trs[0].find_all('th')] # header row
    if headerow: # if there is a header row include first
        rows.append(headerow)
        trs = trs[1:]
    for tr in trs: # for every table row
        link = []
        rows.append([td.get_text(strip=True) for td in tr.find_all('td')]) 
        link = [link['href'] for link in soup.find_all('a', href=True)] #creating list of href value
        del link[-1]
        df = pd.DataFrame(rows)  # creating anda dataframe from list 
        df.columns = df.iloc[0] # Creating New Column Name
        df.drop(df.index[0], inplace=True)
        df.drop(['View Details','View Application',	'View Certificate'],axis=1, inplace = True)
        df['View Link'] = pd.Series(link)
    return df

## Python program to navigate webpages on https://maharerait.mahaonline.gov.in/ and gather information of Registered Agent in Mumbai City.

In [171]:
from selenium.webdriver.support.ui import WebDriverWait
import time
driver.get("https://maharerait.mahaonline.gov.in/Login/Login");
driver.find_element_by_link_text('Search Project Details').click()
driver.find_element_by_id('Agent').click()
driver.find_element_by_id("btnAdvance").click()
dropdown1 = Select(driver.find_element_by_id('State'))
dropdown1.select_by_value("27")
time.sleep(1)
dropdown2 = Select(driver.find_element_by_id('District'))
time.sleep(2)
dropdown2.select_by_visible_text('Mumbai City')
driver.find_element_by_id("btnSearch").click()
print(driver.title)
#driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
time.sleep(2)
my_df  = pd.DataFrame() # creating empty dataframe
x =0
while x < 50: # Gathering Info from First 50 pages out of total 180 pages
  #x =0
  soup=BeautifulSoup(driver.page_source, 'html.parser')
  htmltable = soup.find('table', { 'class' : 'table table-striped grid-table' })
  list_table = tableDataText(htmltable)
  #df = df.append(data_link)
  my_df = my_df.append(list_table)
  time.sleep(3)
  x+=1
  driver.find_element_by_id("btnNext").click()


driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
driver.save_screenshot("image.png")
image = Image.open("image.png")
image.show()
 # Close the browser instance


Login Page : Maharashtra Real Estate Regulatory Authority


In [ ]:
driver.quit()

In [172]:
my_df.tail(10)

,Sr No.,Agent Name,Certificate No.,View Link
41,2491,SAURABH KULKARNI,A51900027350,/PrintPreview/PrintPreview?q=c1%2fjG8%2fMomd4Q...
42,2492,Jatin Pravinchandra Shah,A51900005505,/PrintPreview/PrintPreview?q=gRYTUqqYynwIu5vH1...
43,2493,YELLOW KEY,A51900003400,/PrintPreview/PrintPreview?q=qhFeM8WraAUKFKMGB...
44,2494,BHONESH KEWAL BHOJWANI,A51900027246,/PrintPreview/PrintPreview?q=%2f3QEwhT8DN5U5dR...
45,2495,AJIT JANARDAN RANE,A51900027348,/PrintPreview/PrintPreview?q=dRhh4%2f%2foMy2m9...
46,2496,NILESH HIRACHAND JHAVERI,A51900027383,/PrintPreview/PrintPreview?q=gOrH8ML0ita7yEqls...
47,2497,Manish Jagdish Chheda,A51900002826,/PrintPreview/PrintPreview?q=HUUe2rJccEI%2bdCn...
48,2498,Poonam Prakash Sharma,A51900027536,/PrintPreview/PrintPreview?q=9roksKxUqegZNqyQv...
49,2499,Vijaykumar Rajendra Singh,A51900027347,/PrintPreview/PrintPreview?q=in3avXC0SCX8C8wYz...
50,2500,SHLOKH SANJAY SANGTANI,A51900027456,NaN


In [173]:
my_df.dropna(subset = ["View Link"], inplace=True) #Droing rows with NaN value

In [174]:
my_df['View Link'] = my_df['View Link'].apply(lambda x: 'https://maharerait.mahaonline.gov.in{0}'.format(x))

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
print(my_df.head(3))

In [183]:
#Saving Dataframe into csv format
my_df.to_csv('Agentsinfo.csv', header=True, index=False)


In [184]:
Agentinfo = pd.read_csv('/content/Agentsinfo.csv')

In [185]:
Agentinfo.head(10)

,Sr No.,Agent Name,Certificate No.,View Link
0,1,anisha aziz kabani,A51900013862,https://maharerait.mahaonline.gov.in/PrintPrev...
1,2,SATISSH AGRAWAL,A51900013631,https://maharerait.mahaonline.gov.in/PrintPrev...
2,3,Aditya Sitaram Kumar,A51900013938,https://maharerait.mahaonline.gov.in/PrintPrev...
3,4,Manoj Madhusudan Ranadive,A51900013652,https://maharerait.mahaonline.gov.in/PrintPrev...
4,5,NARAYAN KARWA,A51900021101,https://maharerait.mahaonline.gov.in/PrintPrev...
5,6,ANILKUMAR NANDLAL MORARKA,A51900014131,https://maharerait.mahaonline.gov.in/PrintPrev...
6,7,JAYANT JAGANNATH SIRSAT,A51900013617,https://maharerait.mahaonline.gov.in/PrintPrev...
7,8,RANJIT JAYANTILAL CHOUHAN,A51900013735,https://maharerait.mahaonline.gov.in/PrintPrev...
8,9,SANTOSH GANPAT GAONKAR,A51900013892,https://maharerait.mahaonline.gov.in/PrintPrev...
9,10,Best India Properties Pvt Ltd,A51900013921,https://maharerait.mahaonline.gov.in/PrintPrev...
